In [3]:
import os
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from tqdm import tqdm
from math import log
from fttransformer_mask import FTTransformer

# ============ 設定 ============
DATA_PATH = "cardio_train.csv"
TEST_RATIO = 0.2
categorical_features = ["gender", "cholesterol", "gluc", "smoke", "alco", "active"]
continuous_features = ["age", "height", "weight", "ap_hi", "ap_lo"]
additional_features = ["cholesterol", "gluc", "smoke", "alco", "active"]
MASK_BASE = [False, False] + [True] * 5 + [False] * 5
DEVICE = torch.device("cuda")

# エントロピー計算
def entropy(p: float) -> float:
    if p <= 0 or p >= 1:
        return 0.0
    return - (p * log(p) + (1 - p) * log(1 - p))

# モデル保存パス
fold_to_model_path = {
    1: "./save_mask_models/best_model_epoch300_fold1_ValAcc0.732.pth",
    2: "./save_mask_models/best_model_epoch300_fold2_ValAcc0.739.pth",
    3: "./save_mask_models/best_model_epoch300_fold3_ValAcc0.731.pth",
    4: "./save_mask_models/best_model_epoch300_fold4_ValAcc0.737.pth",
    5: "./save_mask_models/best_model_epoch300_fold5_ValAcc0.735.pth",
}

def main():
    data = pd.read_csv(DATA_PATH, sep=';')
    ids = data["id"].values
    X = data.drop(columns=["cardio"])
    y = data["cardio"]

    X_trainval, X_test, y_trainval, y_test, id_trainval, id_test = \
        train_test_split(X, y, ids, test_size=TEST_RATIO, random_state=42, stratify=y)

    # 前処理：エンコード＆スケール
    encoders = {}
    for col in categorical_features:
        le = LabelEncoder()
        X_trainval[col] = le.fit_transform(X_trainval[col])
        X_test[col] = le.transform(X_test[col])
        encoders[col] = le
    scaler = StandardScaler()
    X_trainval[continuous_features] = scaler.fit_transform(X_trainval[continuous_features])
    X_test[continuous_features] = scaler.transform(X_test[continuous_features])

    # 出現確率の再計算
    feature_value_probs = {
        feat: X_trainval[feat].value_counts(normalize=True).sort_index().to_dict()
        for feat in additional_features
    }

    # 各foldごとに実行
    for fold, model_path in fold_to_model_path.items():
        print(f"=== Fold{fold} モデル読み込み: {model_path} ===")
        if not os.path.exists(model_path):
            print(f"❌ モデルが存在しません: {model_path}")
            continue

        # モデルロード
        model = FTTransformer(
            categories=[X[cat].nunique() for cat in categorical_features],
            num_continuous=len(continuous_features),
            dim=64, depth=6, heads=8, ff_dropout=0.2, attn_dropout=0.2
        ).to(DEVICE)
        model.load_state_dict(torch.load(model_path, map_location=DEVICE))
        model.eval()

        results = []
        for i in tqdm(range(len(X_test)), desc=f"Fold{fold} 推論", unit="件"):
            sid = int(id_test[i])
            label = int(y_test.iloc[i])
            x_cat = torch.tensor([X_test.iloc[i][categorical_features].values], dtype=torch.long).to(DEVICE)
            x_cont = torch.tensor([X_test.iloc[i][continuous_features].values.astype(np.float32)],
                                dtype=torch.float32).to(DEVICE)

            # ベース推論＆エントロピー
            with torch.no_grad():
                base_conf = torch.sigmoid(model(x_cat, x_cont, torch.tensor([MASK_BASE],dtype=torch.bool).to(DEVICE))).item()
            base_ent = entropy(base_conf)
            base_pred = int(base_conf > 0.5)

            # エントロピー削減量で特徴選択
            best_red = -1.0
            best_feat = None
            for feat in additional_features:
                mask = MASK_BASE.copy()
                mask[2 + additional_features.index(feat)] = False
                exp_ent = 0.0
                for val, prob in feature_value_probs[feat].items():
                    x_alt = x_cat.clone()
                    x_alt[0, categorical_features.index(feat)] = int(val)
                    with torch.no_grad():
                        conf_alt = torch.sigmoid(model(x_alt, x_cont, torch.tensor([mask],dtype=torch.bool).to(DEVICE))).item()
                    exp_ent += entropy(conf_alt) * prob
                red = base_ent - exp_ent
                if red > best_red:
                    best_red = red
                    best_feat = feat

            # 改良後推論
            final_mask = MASK_BASE.copy()
            final_mask[2 + additional_features.index(best_feat)] = False
            with torch.no_grad():
                best_conf = torch.sigmoid(model(x_cat, x_cont, torch.tensor([final_mask],dtype=torch.bool).to(DEVICE))).item()
            imp_pred = int(best_conf > 0.5)

            results.append({
                'id': sid,
                'answer': label,
                'base_pred': base_pred,
                'improved_pred': imp_pred,
                'chosen_feat': best_feat,
            })

        # 混同行列プロット（Blues）
        y_true = [r['answer'] for r in results]
        cm_base = confusion_matrix(y_true, [r['base_pred'] for r in results])
        cm_imp  = confusion_matrix(y_true, [r['improved_pred'] for r in results])

        disp = ConfusionMatrixDisplay(cm_base, display_labels=[0,1])
        disp.plot(cmap='Blues')
        plt.title(f"Fold{fold} Base CM")
        plt.savefig(f"fold{fold}_base_cm.png")
        plt.close()

        disp = ConfusionMatrixDisplay(cm_imp, display_labels=[0,1])
        disp.plot(cmap='Blues')
        plt.title(f"Fold{fold} Improved CM")
        plt.savefig(f"fold{fold}_improved_cm.png")
        plt.close()

        # CSV保存
        pd.DataFrame(results).sort_values('id').to_csv(f"fold{fold}_results.csv", index=False)
        print(f"✅ fold{fold}_results.csv saved\n")

if __name__ == "__main__":
    main()

=== Fold1 モデル読み込み: ./save_mask_models/best_model_epoch300_fold1_ValAcc0.732.pth ===


Fold1 推論: 100%|██████████| 14000/14000 [17:06<00:00, 13.64件/s]


✅ fold1_results.csv saved

=== Fold2 モデル読み込み: ./save_mask_models/best_model_epoch300_fold2_ValAcc0.739.pth ===


Fold2 推論:  21%|██        | 2932/14000 [03:37<13:39, 13.51件/s]


KeyboardInterrupt: 